In [1]:
import os
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

Path: 경로를 운영체제에 안전하게 다루기 위한 도구.

torch, nn: 모델/학습/텐서 연산.

DataLoader: 배치로 데이터를 공급.

WeightedRandomSampler: **클래스 불균형(역주행 적음)**을 보정하기 위한 샘플러.

torchvision.datasets.ImageFolder: dataset/train/normal 같은 폴더 구조를 자동으로 데이터셋으로 읽음.

transforms: 이미지 전처리/증강.

models: ResNet 같은 사전학습 모델 로드.

classification_report, confusion_matrix: 테스트 성능을 보기 위한 지표.

tqdm: 학습 진행바.

In [2]:
DATA_DIR = Path("dataset")
BATCH_SIZE = 32
EPOCHS = 8
LR = 1e-3
IMG_SIZE = 224


DATA_DIR: dataset/train, dataset/val, dataset/test 폴더가 있는 위치.

BATCH_SIZE=32: 한 번에 32장씩 학습.

EPOCHS=8: train 전체를 8번 반복.

LR=1e-3: 학습률(FC만 학습이라 이 정도가 보통 무난).

IMG_SIZE=224: ResNet 기본 입력 크기.

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cpu


GPU 사용 가능하면 cuda, 아니면 cpu.

출력으로 확인.

4) 이미지 전처리/증강 정의

In [4]:
#(1) train용 증강

In [9]:
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.05),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])


Resize(224,224): 입력 크기를 통일.

ColorJitter(...): 밝기/대비/채도/색조를 랜덤 변화 → 조명/시간대 다양성에 강해짐.

RandomApply(GaussianBlur, p=0.2): 20% 확률로 블러 → 흔들림/저화질에 강해짐.

ToTensor(): PIL 이미지를 텐서로 변환(0~1 범위).

Normalize(...): ImageNet 평균/표준편차로 정규화
→ ImageNet 사전학습 모델에 맞는 입력 분포로 맞추는 핵심.


In [ ]:
# (2) val/test용 전처리(증강 없음)
# 평가 데이터는 랜덤성 없이 항상 동일하게 처리해야 점수 비교가 안정적임.

In [10]:
eval_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [11]:
# 5) 데이터셋 로드(ImageFolder)
train_ds = datasets.ImageFolder(DATA_DIR/"train", transform=train_tf)
val_ds   = datasets.ImageFolder(DATA_DIR/"val",   transform=eval_tf)
test_ds  = datasets.ImageFolder(DATA_DIR/"test",  transform=eval_tf)

print("classes:", train_ds.classes)
num_classes = len(train_ds.classes)

classes: ['normal', 'wrongway']


ImageFolder는 폴더명을 클래스 라벨로 씀.

dataset/train/normal/*.jpg

dataset/train/wrongway/*.jpg

train_ds.classes는 보통 ['normal', 'wrongway'] (알파벳 순).

num_classes는 2.

In [12]:
# 6) 불균형 처리: WeightedRandomSampler 
# (핵심 : 역주행의 현실적인 적용, 클래스 빈도 기반 가중치 부여)

targets = [y for _, y in train_ds.samples]
class_count = torch.bincount(torch.tensor(targets))
class_weight = 1.0 / class_count.float()
sample_weight = [class_weight[t].item() for t in targets]
sampler = WeightedRandomSampler(sample_weight, num_samples=len(sample_weight), replacement=True)


train_ds.samples는 (이미지경로, 라벨id) 리스트.

targets: 라벨만 뽑아옴.

class_count: 각 클래스 개수 세기. (예: normal 840, wrongway 43)

class_weight = 1 / class_count: 적은 클래스(역주행)에 더 큰 가중치.

sample_weight: 각 샘플에 해당 클래스 가중치를 부여.

WeightedRandomSampler: 이 가중치에 따라 샘플을 “복원추출(replacement=True)”로 뽑음

In [13]:
# 7) DataLoader 생성
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

train은 shuffle 대신 sampler를 사용(둘은 같이 못 씀).

val/test는 순서 상관 없어서 shuffle=False.

num_workers=0: Windows에서 멀티프로세싱 이슈 피하기

In [15]:
# 8) 모델 준비: ResNet-18 전이학습

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
for p in model.parameters():
    p.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

requires_grad=False: 백본(특징추출기)을 고정(freeze)
→ 데이터가 적을 때 과적합 감소 + 학습 속도 증가.

model.fc 교체: 원래 1000클래스 분류기 → 2클래스 분류기로 바꿈.

to(device): GPU/CPU로 모델 이동.

In [16]:
# 9) 손실함수와 최적화
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=LR)

CrossEntropyLoss: 다중분류 표준 손실(2클래스도 여기에 포함).

옵티마이저는 model.fc.parameters()만 학습
→ 백본은 freeze라 업데이트 안 됨.

In [17]:
# 10) 한 epoch 돌리는 함수 run_epoch
def run_epoch(loader, train=False):
    model.train(train)
    total_loss, correct, total = 0.0, 0, 0

    for x, y in tqdm(loader, leave=False):
        x, y = x.to(device), y.to(device)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            logits = model(x)
            loss = criterion(logits, y)

            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss/total, correct/total



함수는 loader 한 바퀴를 돌면서:

train=True면 학습 모드 + 역전파 수행

train=False면 평가 모드(가중치 업데이트 없음)

세부 포인트:

model.train(train)

train=True → dropout/bn 등이 학습 모드

train=False → 평가 모드로 고정

torch.set_grad_enabled(train)

평가 때는 gradient 계산을 꺼서 속도↑/메모리↓

loss.item() * x.size(0)

배치 평균 loss를 샘플 수로 되돌려 누적 → epoch 평균 계산 정확

pred = logits.argmax(dim=1)

가장 점수가 높은 클래스를 예측 클래스로 선택

반환: (epoch 평균 loss, epoch accuracy)

In [18]:
# 11) 학습 루프 + best 모델 저장
best_val = 0.0
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader, train=False)
    print(...)

    if va_acc > best_val:
        best_val = va_acc
        torch.save(model.state_dict(), "best.pt")


Ellipsis


Ellipsis


Ellipsis


Ellipsis


Ellipsis


Ellipsis


Ellipsis


Ellipsis


epoch마다 train/val 성능 측정.

val acc가 최고일 때의 모델만 best.pt로 저장.

과적합을 어느 정도 방지(“가장 잘 일반화된 시점” 저장)

In [19]:
# 12) 테스트 평가
model.load_state_dict(torch.load("best.pt", map_location=device))
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for x, y in tqdm(test_loader, leave=False):
        x = x.to(device)
        logits = model(x)
        pred = logits.argmax(dim=1).cpu().tolist()
        y_pred += pred
        y_true += y.tolist()

print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=test_ds.classes))


Confusion matrix:
 [[177   3]
 [  2   8]]
              precision    recall  f1-score   support

      normal       0.99      0.98      0.99       180
    wrongway       0.73      0.80      0.76        10

    accuracy                           0.97       190
   macro avg       0.86      0.89      0.87       190
weighted avg       0.98      0.97      0.97       190



저장해둔 best.pt를 다시 로드해서 테스트.

model.eval() + torch.no_grad()
→ 평가 안정 + 속도/메모리 절약.

confusion_matrix: 정상/역주행 각각 맞춘/틀린 개수를 표로 보여줌.

classification_report: precision/recall/f1/support 출력.

여기서 네 프로젝트 핵심은 보통 **wrongway recall(역주행을 얼마나 놓치지 않는가)**야.

In [ ]:
# 이 코드에서 “핵심 설계 포인트 3개”

# ImageNet 사전학습 ResNet-18 → 작은 데이터에서도 학습 가능

# 백본 freeze + FC만 학습 → 과적합 줄이고 빠른 MVP

# WeightedRandomSampler → 역주행(소수 클래스)을 학습에서 충분히 보게 함